# United outdoors datawarehouse

## Imports

In [19]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import OperationalError
import urllib
import re

## Database connection details

In [20]:
DB = {
    'servername' : '(local)\\SQLEXPRESS',
    'united_outdoors_database' : 'UnitedOutdoors',
    'northwind_database' : 'Northwind',
    'aenc_database' : 'Aenc',
    'adventureworks_database' : 'AdventureWorks2019',
    'master' : 'master'
}

## Create the datawarehouse

In [21]:
creation_params = urllib.parse.quote_plus(f'DRIVER={{SQL Server}};SERVER={DB["servername"]};DATABASE={DB["master"]};Trusted_Connection=yes')

creation_engine = create_engine(f'mssql+pyodbc:///?odbc_connect={creation_params}')

# Open the SQL script file and read its contents
with open('sql/UnitedOutdoors_creation.sql', 'r') as file:
    sql_script = file.read()

# splitting the script into the database creation and the rest
commands = re.split(r'GO\n', sql_script)

# removing all \bGO\b from the commands
commands = [re.sub(r'\bGO\b', '', command) for command in commands]

# Execute the commands
for command in commands:
    command = command.strip()
    # Skip if the command is empty or 'GO'
    if not command or command.upper() == 'GO':
        continue
    try:
        conn = creation_engine.connect()
        cursor = conn.connection.cursor()
        cursor.execute(command)
        conn.connection.commit()
    except OperationalError as e:
        print(f'Error: {e} at command: {command}')
    finally:
        cursor.close()
        conn.close()
        creation_engine.dispose()

## Connecting to the UnitedOutdoors datawarehouse

In [22]:
params = urllib.parse.quote_plus(f'DRIVER={{SQL Server}};SERVER={DB["servername"]};DATABASE={DB["united_outdoors_database"]};Trusted_Connection=yes')

united_outdoors_engine = create_engine(f'mssql+pyodbc:///?odbc_connect={params}')

try:
    united_outdoors_conn = united_outdoors_engine.connect()
    print('Connection to United Outdoors database successful')
except OperationalError as e:
    print(f'Error: {e}')

Connection to United Outdoors database successful


## Loading the data from the source databases

### Northwind database

In [23]:
params = urllib.parse.quote_plus(f'DRIVER={{SQL Server}};SERVER={DB["servername"]};DATABASE={DB["northwind_database"]};Trusted_Connection=yes')

northwind_engine = create_engine(f'mssql+pyodbc:///?odbc_connect={params}')

try:
    northwind_conn = northwind_engine.connect()
    print('Connection to NorthWind database successful')
except OperationalError as e:
    print(f'Error: {e}')

Connection to NorthWind database successful


### Aenc database

In [24]:
params = urllib.parse.quote_plus(f'DRIVER={{SQL Server}};SERVER={DB["servername"]};DATABASE={DB["aenc_database"]};Trusted_Connection=yes')

aenc_engine = create_engine(f'mssql+pyodbc:///?odbc_connect={params}')

try:
    aenc_engine = aenc_engine.connect()
    print('Connection to Aenc database successful')
except OperationalError as e:
    print(f'Error: {e}')

Connection to Aenc database successful


### AdventureWorks database

In [25]:
params = urllib.parse.quote_plus(f'DRIVER={{SQL Server}};SERVER={DB["servername"]};DATABASE={DB["adventureworks_database"]};Trusted_Connection=yes')

adventureworks_engine = create_engine(f'mssql+pyodbc:///?odbc_connect={params}')

try:
    adventureworks_conn = adventureworks_engine.connect()
    print('Connection to AdventureWorks database successful')
except OperationalError as e:
    print(f'Error: {e}')

Connection to AdventureWorks database successful


# Closing connections

In [26]:
united_outdoors_conn.close()
northwind_conn.close()
aenc_engine.close()
adventureworks_conn.close()